In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from tqdm import tqdm

In [2]:
comments=pd.read_csv('./final.csv')

In [3]:
comments.shape

(346959, 1)

In [4]:
comments.isnull().sum()

comments    0
dtype: int64

In [5]:
comments.head()

,comments
0,Stephanie was a wonderful host! Her apartment ...
1,Such a wonderful place and very close to the m...
2,I just got back from a trip to NYC during whic...
3,Stephanie's offered all the most important thi...
4,"Stephanie was really nice, ftiendly and helpfu..."


In [6]:
comments.dropna(inplace=True)

In [7]:
comments.isnull().sum()

comments    0
dtype: int64

In [8]:
import re
import numpy as np

In [9]:
import gensim

In [10]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [11]:
import spacy

In [12]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ikhyvicky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
# Convert email body to list
data = comments.comments.values.tolist()

In [16]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [17]:
data_words=list(sent_to_words(data))

In [18]:
print(data_words[3])

['stephanie', 'offered', 'all', 'the', 'most', 'important', 'things', 'warm', 'welcome', 'into', 'comfortable', 'home', 'comfortable', 'bed', 'in', 'quiet', 'room', 'fresh', 'clean', 'towels', 'blankets', 'and', 'easy', 'access', 'to', 'manhattan', 'finding', 'myself', 'travelling', 'to', 'nyc', 'in', 'the', 'future', 'feel', 'already', 'have', 'open', 'invitation', 'to', 'make', 'home', 'away', 'from', 'home', 'through', 'stephanie', 'generousity']


In [19]:
from gensim.models.phrases import Phrases, Phraser

In [20]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

In [21]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [22]:
print(trigram_mod[bigram_mod[data_words[35]]])

['best', 'host', 'ever', 'had', 'stefanie', 'and', 'her', 'family', 'are', 'very', 'kind', 'helpful', 'and', 'accommodating', 'you', 'will', 'feel', 'at', 'home', 'here', 'immediately', 'the', 'place', 'is', 'very', 'nice', 'clean', 'and', 'you', 'can', 'beat', 'the', 'access', 'to', 'both', 'the', 'and', 'as', 'well', 'as', 'the', 'and', 'subway', 'lines', 'both', 'each', 'blocks', 'away', 'will', 'definitely', 'come', 'back']


In [23]:
#remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
data_words_nostops = remove_stopwords(data_words)

In [25]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [26]:
import en_core_web_sm

In [27]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = en_core_web_sm.load(disable=['parser', 'ner'])

In [28]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [29]:
import pickle

In [30]:
f=open("./model/data_lemmatized.pkl",'wb')
pickle.dump(data_lemmatized,f)
f.close()

In [ ]:
#data_lemmatized = []
#with (open("data_lemmatized.pkl", "rb")) as openfile:
    #while True:
        #try:
        #    data_lemmatized.append(pickle.load(openfile))
        #except EOFError:
        #    break

In [17]:
#data_lemmatized=data_lemmatized[0]

In [31]:
len(data_lemmatized)

346959

In [32]:
#create a dictionary and corpus for (LDA topic modeling)

#creating the dictionary

idtoword=corpora.Dictionary(data_lemmatized)
#create corpus

texts=data_lemmatized

corpus=[idtoword.doc2bow(text) for text in texts]

In [33]:
len(corpus)

346959

# Gensim LDA Model -Topic Modeling

In [34]:
#build our LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=idtoword,
                                           num_topics=10,
                                           workers=3,
                                           random_state=100,
                                           eval_every=1,
                                           chunksize=100,
                                           passes=10)


In [137]:
f=open("./model/lda.pkl",'wb')
pickle.dump(lda_model,f)
f.close()

In [138]:
g=open("./model/idtoword.pkl",'wb')
pickle.dump(idtoword,g)
g.close()

In [139]:
h=open("./model/corpus.pkl",'wb')
pickle.dump(corpus,h)
h.close()

In [68]:
#pickle.dump(lda_model, open("./model/lda.pickle", "wb"))
#pickle.dump(idtoword, open("./model/idatoword.pickle", "wb"))
#pickle.dump(corpus, open("./model/corpus.pickle", "wb"))


In [38]:
print(lda_model.print_topics())

[(0, '0.054*"place" + 0.039*"nice" + 0.038*"minute" + 0.038*"close" + 0.038*"subway" + 0.031*"walk" + 0.031*"station" + 0.028*"really" + 0.026*"stay" + 0.025*"good"'), (1, '0.045*"host" + 0.040*"day" + 0.024*"arrival" + 0.021*"reservation" + 0.021*"cancel" + 0.015*"door" + 0.014*"apartment" + 0.012*"floor" + 0.011*"say" + 0.010*"airbnb"'), (2, '0.048*"stay" + 0.034*"place" + 0.027*"home" + 0.025*"feel" + 0.024*"make" + 0.022*"back" + 0.021*"come" + 0.020*"time" + 0.017*"thank" + 0.016*"definitely"'), (3, '0.068*"question" + 0.062*"respond" + 0.044*"always" + 0.042*"quick" + 0.041*"quickly" + 0.035*"answer" + 0.030*"message" + 0.023*"describe" + 0.023*"transportation" + 0.023*"help"'), (4, '0.098*"check" + 0.041*"easy" + 0.028*"make" + 0.028*"stay" + 0.021*"get" + 0.018*"late" + 0.018*"time" + 0.018*"need" + 0.015*"accommodate" + 0.014*"night"'), (5, '0.069*"good" + 0.040*"location" + 0.029*"place" + 0.029*"apartment" + 0.027*"night" + 0.023*"price" + 0.018*"sleep" + 0.017*"noise" + 0.0

In [39]:
lda_model.save('lda.model')

In [40]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, idtoword)
pyLDAvis.display(vis)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [41]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))



Perplexity:  -6.8720646171631525


In [42]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=idtoword, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy has detected that you (may be) writing to an array with
overlapping memory from np.broadcast_arrays. If this is intentional
set the WRITEABLE flag True or make a copy immediately before writing.
  i, j, x)
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy ha


Coherence Score:  0.597899418023428


In [43]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [44]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

In [45]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [46]:
df_dominant_topic.shape

(346959, 5)

In [47]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.7248,"great, place, location, stay, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...
1,1,2.0,0.3116,"stay, place, home, feel, make, back, come, tim...",Such a wonderful place and very close to the m...
2,2,2.0,0.4839,"stay, place, home, feel, make, back, come, tim...",I just got back from a trip to NYC during whic...
3,3,2.0,0.4605,"stay, place, home, feel, make, back, come, tim...",Stephanie's offered all the most important thi...
4,4,0.0,0.5278,"place, nice, minute, close, subway, walk, stat...","Stephanie was really nice, ftiendly and helpfu..."
5,5,2.0,0.7294,"stay, place, home, feel, make, back, come, tim...",I was pleasantly surprised with my whole stay ...
6,6,0.0,0.6768,"place, nice, minute, close, subway, walk, stat...",Stephanie's apt was great. Very convenient to ...
7,7,2.0,0.2785,"stay, place, home, feel, make, back, come, tim...",my husband and i had such a great time staying...
8,8,2.0,0.3413,"stay, place, home, feel, make, back, come, tim...","When I first arrived at Stephanie's, she was v..."
9,9,5.0,0.5499,"good, location, place, apartment, night, price...",awesome couldn't have been better.


In [48]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [49]:
sia = SentimentIntensityAnalyzer()

In [50]:
example=df_dominant_topic.loc[:10,:]

In [51]:
example.shape

(11, 5)

In [52]:
for sentence in example.Text:
    # Assess sentiment.
    sentiment = sia.polarity_scores(sentence)["compound"]
    print(sentence)
    print(sentiment)
    print('\n')

Stephanie was a wonderful host! Her apartment was beautiful and extremely clean. We had a very nice (and affordable!) stay in the city with private room and own bathroom.
0.9357


Such a wonderful place and very close to the metro station on the line 1 2 3. Your She was awesome with us, It was a pleasure to stay in her appt. she's french, so last word ...
Stephanie thank you again for your hospitality. We had a great time with you. Very available for us with lots of advice, I would particularly recommend this address which is more for a reasonable price. The subway is just 1 block from the apartment and the 1-2-3 line is the central and most convenient NYC. her neighborhood is really very nice and more !!! Go there with closed eyes.
0.9727


I just got back from a trip to NYC during which I stayed at Stephanie's appartment. The bedroom and private bathroom that comes with it were both kept very clean and are spacious so that I could unpack my stuff and put my clothes away in the closet

In [53]:
sentiment_scores=[]
for sentence in df_dominant_topic.Text:
    sentiment = sia.polarity_scores(sentence)["compound"]
    sentiment_scores.append(sentiment)
    

In [54]:
df_dominant_topic['sentiment_score'] = pd.Series(sentiment_scores, index=df_dominant_topic.index)

In [62]:
df_dominant_topic.shape

(346959, 6)

In [55]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,9.0,0.7248,"great, place, location, stay, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,2.0,0.3116,"stay, place, home, feel, make, back, come, tim...",Such a wonderful place and very close to the m...,0.9727
2,2,2.0,0.4839,"stay, place, home, feel, make, back, come, tim...",I just got back from a trip to NYC during whic...,0.9620
3,3,2.0,0.4605,"stay, place, home, feel, make, back, come, tim...",Stephanie's offered all the most important thi...,0.9626
4,4,0.0,0.5278,"place, nice, minute, close, subway, walk, stat...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [56]:
#to get a sense of what negative comments will look like
negative_sentiments=df_dominant_topic[df_dominant_topic['sentiment_score']<0]

In [57]:
negative_sentiments.reset_index(drop=True,inplace=True)

In [58]:
negative_sentiments.shape

(8996, 6)

In [59]:
negative_sentiments.sort_values('sentiment_score').head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
3565,162817,1.0,0.4476,"host, day, arrival, reservation, cancel, door,...",This is a filthy hole in a filthy old building...,-0.9971
2277,111046,1.0,0.3777,"host, day, arrival, reservation, cancel, door,...",Dangerous host \nI really hate it when I write...,-0.9966
8788,338855,1.0,0.4437,"host, day, arrival, reservation, cancel, door,...",We had a very bad experience with this Airbnb ...,-0.9965
3748,169226,1.0,0.4290,"host, day, arrival, reservation, cancel, door,...",Here is a list of issues to consider. We reach...,-0.9963
7510,297779,1.0,0.4484,"host, day, arrival, reservation, cancel, door,...",I really do not enjoy having to write bad revi...,-0.9963
6872,277427,1.0,0.4358,"host, day, arrival, reservation, cancel, door,...",From the moment me and my sister booked this A...,-0.9956
1572,82528,1.0,0.4002,"host, day, arrival, reservation, cancel, door,...",BAD BAD TERRIEBLE. FROM THE BEGINNING. Very ru...,-0.9949
212,12044,1.0,0.4175,"host, day, arrival, reservation, cancel, door,...","During the stay, Zooey and I felt very happy t...",-0.9948
1024,57391,1.0,0.3901,"host, day, arrival, reservation, cancel, door,...",My wife and 4 kids had a very dissatisfying st...,-0.9947
2526,124547,1.0,0.4017,"host, day, arrival, reservation, cancel, door,...",We have used AirBnb and (Hidden by Airbnb) as ...,-0.9946


In [60]:
negative_sentiments.Text.iloc[8788]

"We had a very bad experience with this Airbnb stay. Zheng was very friendly as a person and tried to help, but there were so many things wrong and illegal about her setup, that it led to the worst Airbnb experience we have ever had. Zheng has put her rental apartment on Airbnb without permission from her leasing office (as she told us herself). Hence, she lied about many things in her listing and to us when we booked. She gave us a wrong  address since she didn't want us to reach her main entrance/lobby or use the main elevator since she thought her leasing agency folks will catch her doing Airbnb illegally.  Further, she asked us not to use the lobby, main elevator and the correct address (anybody for deliveries would then reach the lobby and meet the leasing office people). Instead, she asked us to use a freight elevator on one side of the building to enter the property. This led to a series of bad experiences for us. I travel for business and I book using Airbnb for business. One o

In [143]:
negative_sentiments.Dominant_Topic.nunique() #this just means people  can talk negative things for each topic

20

WHAT I REALISED
<br>the topic classification via LDA although still not the best poppped up few interesting things:
1. reducing no of topics seem to be improving the coherence score

In [64]:
#putting them back together
df=pd.read_csv('./removed_january.csv')

In [67]:
df.dropna(inplace=True)

In [68]:
df.shape

(346959, 6)

In [69]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2515,198,2008-10-13,2603,Jenny,Stephanie was a wonderful host! Her apartment ...
1,2515,859,2009-03-08,8455,Roland,Such a wonderful place and very close to the m...
2,2515,1083,2009-03-25,9759,Cem,I just got back from a trip to NYC during whic...
3,2515,1107,2009-03-27,9193,Holly,Stephanie's offered all the most important thi...
4,2515,2175,2009-05-09,7048,Alessandra,"Stephanie was really nice, ftiendly and helpfu..."


In [82]:
df_dominant_topic.shape

(346959, 5)

In [70]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,9.0,0.7248,"great, place, location, stay, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,2.0,0.3116,"stay, place, home, feel, make, back, come, tim...",Such a wonderful place and very close to the m...,0.9727
2,2,2.0,0.4839,"stay, place, home, feel, make, back, come, tim...",I just got back from a trip to NYC during whic...,0.9620
3,3,2.0,0.4605,"stay, place, home, feel, make, back, come, tim...",Stephanie's offered all the most important thi...,0.9626
4,4,0.0,0.5278,"place, nice, minute, close, subway, walk, stat...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [72]:
#drop original non-translated comments
df.drop(columns=['comments'],axis=1,inplace=True)

In [99]:
df.reset_index(inplace=True)

In [73]:
df_dominant_topic.drop(columns=['Document_No'],axis=1,inplace=True)

In [100]:
#combine the two together 
frames=[df,df_dominant_topic]
finaldf=pd.concat(frames,axis=1)

In [101]:
finaldf.head()

,index,listing_id,id,date,reviewer_id,reviewer_name,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,2515,198,2008-10-13,2603,Jenny,9.0,0.7248,"great, place, location, stay, host, clean, rec...",Stephanie was a wonderful host! Her apartment ...,0.9357
1,1,2515,859,2009-03-08,8455,Roland,2.0,0.3116,"stay, place, home, feel, make, back, come, tim...",Such a wonderful place and very close to the m...,0.9727
2,2,2515,1083,2009-03-25,9759,Cem,2.0,0.4839,"stay, place, home, feel, make, back, come, tim...",I just got back from a trip to NYC during whic...,0.9620
3,3,2515,1107,2009-03-27,9193,Holly,2.0,0.4605,"stay, place, home, feel, make, back, come, tim...",Stephanie's offered all the most important thi...,0.9626
4,4,2515,2175,2009-05-09,7048,Alessandra,0.0,0.5278,"place, nice, minute, close, subway, walk, stat...","Stephanie was really nice, ftiendly and helpfu...",0.9335


In [102]:
finaldf.shape

(346959, 11)

In [103]:
finaldf.to_csv('./finaldf',index=False)

In [105]:
#now the tedious part is to link listing id to negative sentiment scores
final_negative_sentiments=finaldf[finaldf['sentiment_score']<0]

In [108]:
final_negative_sentiments.shape  #8996 comments with negative comments

(8996, 11)

In [109]:
final_negative_sentiments.listing_id.nunique() #3840 unique listings out of 13322 listings are negatively commented  29% of the listings

3840

In [110]:
df.listing_id.nunique() 

13322

In [117]:
sorted(final_negative_sentiments['listing_id'].value_counts(),reverse=True)

[27,
 25,
 22,
 21,
 21,
 21,
 20,
 19,
 18,
 17,
 17,
 17,
 17,
 17,
 17,
 16,
 16,
 15,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6

In [134]:
listings_sentiments=finaldf.groupby(['listing_id','Dominant_Topic']).agg({'index':'count', 'sentiment_score':'mean'})

In [135]:
listings_sentiments.head(20)

index  sentiment_score
listing_id Dominant_Topic                        
2515       0.0                26         0.779492
           1.0                11         0.355109
           2.0                66         0.888386
           3.0                 1         0.835700
           4.0                 4         0.874725
           5.0                 7         0.634971
           6.0                 6         0.715533
           7.0                14         0.901471
           8.0                13         0.788431
           9.0                27         0.895285
2539       0.0                 4         0.584750
           2.0                 2         0.943450
           8.0                 2         0.699100
           9.0                 1         0.624900
3330       0.0                 5         0.892320
           1.0                 4         0.486350
           2.0                12         0.976725
           4.0                 2         0.683750
           7.0                 2         0.975350
           8.0                 4         0.975200

In [136]:
listings_sentiments[['sentiment_score']<0]

TypeError: '<' not supported between instances of 'list' and 'int'

In [121]:
listings_sentiments.to_csv('./listings_sentiments.csv')